In [1]:
import pandas as pd
import numpy as np

!pip install fuzzywuzzy python-Levenshtein
from fuzzywuzzy import process

In [2]:
df = pd.read_csv('fact_loan.csv')

In [3]:
df.shape

(3520, 7)

In [4]:
df.head()

,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud
0,1,4015,Business,558811.91,2022-04-21,Default,0
1,2,4007,Education,370193.71,2014-05-28,Closed,0
2,3,4028,Business,344548.54,2025-09-29,Active,0
3,4,4035,Home,692764.55,2015-01-13,Closed,0
4,5,4040,Education,746639.94,2025-01-07,Closed,0


In [5]:
df.isnull().sum()

loan_id        0
customer_id    0
loan_type      0
loan_amount    0
issue_date     0
loan_status    0
is_fraud       0
dtype: int64

In [6]:
df.dtypes

loan_id         int64
customer_id     int64
loan_type      object
loan_amount    object
issue_date     object
loan_status    object
is_fraud       object
dtype: object

In [7]:
df.duplicated().sum()

np.int64(20)

In [8]:
df.drop_duplicates()

,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud
0,1,4015,Business,558811.91,2022-04-21,Default,0
1,2,4007,Education,370193.71,2014-05-28,Closed,0
2,3,4028,Business,344548.54,2025-09-29,Active,0
3,4,4035,Home,692764.55,2015-01-13,Closed,0
4,5,4040,Education,746639.94,2025-01-07,Closed,0
...,...,...,...,...,...,...,...
3495,3496,4036,Vehicle,325198.6,"Aug 26, 2016",Closed,0
3496,3497,4020,Education,856410.24,2016-09-22,Active,0
3497,3498,4022,Gold,162079.85,2013/06/20,Active,0
3498,3499,4021,Home,623729.6,2019-03-29,Default,0


In [9]:
# 1. Fix issue_date - convert to datetime
df['issue_date'] = pd.to_datetime(df['issue_date'], format='mixed', dayfirst=True)

In [10]:
df['issue_date'] = pd.to_datetime(df['issue_date'], errors='coerce')
df.dtypes

loan_id                 int64
customer_id             int64
loan_type              object
loan_amount            object
issue_date     datetime64[ns]
loan_status            object
is_fraud               object
dtype: object

In [11]:
df.isnull().sum()

loan_id        0
customer_id    0
loan_type      0
loan_amount    0
issue_date     0
loan_status    0
is_fraud       0
dtype: int64

In [12]:
df['loan_type'] = (df['loan_type'].astype(str).str.strip().str.lower())
    

In [13]:
# 2. Correct categories
valid_loan_types = [
    'business',
    'education',
    'home',
    'vehicle',
    'gold',
    'personal'
]

In [14]:
# 3. Fuzzy match function
def correct_loan_type(x):
    match, score = process.extractOne(x, valid_loan_types)
    return match if score >= 60 else x

In [15]:
# 4. Apply clean
df['loan_type'] = df['loan_type'].apply(correct_loan_type)


In [16]:
#5. Format properly
df['loan_type'] = df['loan_type'].replace({
    'business': 'Business',
    'education': 'Education',
    'home': 'Home',
    'vehicle': 'Vehicle',
    'gold': 'Gold',
    'personal': 'Personal'
})

In [17]:
print(df['loan_type'].value_counts())
print(df['loan_type'].unique())
df.dtypes

loan_type
Business     609
Education    602
Personal     589
Home         583
Vehicle      577
Gold         560
Name: count, dtype: int64
['Business' 'Education' 'Home' 'Vehicle' 'Gold' 'Personal']


loan_id                 int64
customer_id             int64
loan_type              object
loan_amount            object
issue_date     datetime64[ns]
loan_status            object
is_fraud               object
dtype: object

In [18]:
# Basic clean: remove spaces + lowercase
df['loan_status'] = df['loan_status'].astype(str).str.strip().str.lower()


In [19]:
# Fix to correct categories
df['loan_status'] = df['loan_status'].replace({
    'active': 'Active',
    'closed': 'Closed',
    'default': 'Default'
})

In [20]:
print(df['loan_status'].value_counts())
print(df['loan_status'].unique())


loan_status
Closed     1190
Default    1187
Active     1143
Name: count, dtype: int64
['Default' 'Closed' 'Active']


In [21]:
df.head(373)

,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud
0,1,4015,Business,558811.91,2022-04-21,Default,0
1,2,4007,Education,370193.71,2014-05-28,Closed,0
2,3,4028,Business,344548.54,2025-09-29,Active,0
3,4,4035,Home,692764.55,2015-01-13,Closed,0
4,5,4040,Education,746639.94,2025-01-07,Closed,0
...,...,...,...,...,...,...,...
368,369,4040,Education,178550.38,2015-05-31,Active,0
369,370,4028,Personal,648216.39,2023-10-08,Default,0
370,371,4036,Vehicle,84719.13,2014-06-25,Active,0
371,372,4002,Personal,425344.53,2023-03-24,Default,0


In [22]:
# 5. Fix amount - remove currency symbols and commas
df['loan_amount'] = df['loan_amount'].astype(str).str.replace('₹', '').str.replace(',', '')
df['loan_amount'] = df['loan_amount'].astype(float)
df.dtypes

loan_id                 int64
customer_id             int64
loan_type              object
loan_amount           float64
issue_date     datetime64[ns]
loan_status            object
is_fraud               object
dtype: object

In [24]:
df['is_fraud'] = df['is_fraud'].astype(str).str.strip().str.lower()

In [25]:
#(Handles values: “yes/no”, “true/false”, “1/0”, mixed spellings)
df['is_fraud'] = df['is_fraud'].replace({
    'yes': 1,
    'y': 1,
    'true': 1,
    '1': 1,
    'fraud': 1,
    'no': 0,
    'n': 0,
    'false': 0,
    '0': 0,
    'not fraud': 0
})

C:\Users\swath\AppData\Local\Temp\ipykernel_332\2181811835.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_fraud'] = df['is_fraud'].replace({


In [26]:
df['is_fraud'] = df['is_fraud'].astype(int)
df.dtypes

loan_id                 int64
customer_id             int64
loan_type              object
loan_amount           float64
issue_date     datetime64[ns]
loan_status            object
is_fraud                int64
dtype: object

In [27]:
df.isnull().sum()

loan_id        0
customer_id    0
loan_type      0
loan_amount    0
issue_date     0
loan_status    0
is_fraud       0
dtype: int64

In [28]:
df.to_csv(r"C:\Users\swath\Downloads\loan.csv" , index=False)

In [29]:
df.shape 

(3520, 7)

In [29]:
df.duplicated('loan_id').sum()


np.int64(20)

In [30]:
df[df.duplicated('loan_id', keep=False)].sort_values('loan_id')


,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud
176,177,4035,Business,526196.11,2019-09-29,Closed,0
3515,177,4035,Business,526196.11,2019-09-29,Closed,0
3519,394,4038,Gold,-165622.90,2021-01-26,Active,0
393,394,4038,Gold,-165622.90,2021-01-26,Active,0
438,439,4003,Business,650146.53,2025-09-16,Closed,0
3503,439,4003,Business,650146.53,2025-09-16,Closed,0
490,491,4024,Business,715386.03,2021-02-20,Active,0
3514,491,4024,Business,715386.03,2021-02-20,Active,0
3512,957,4028,Home,343422.86,2014-04-17,Closed,0
956,957,4028,Home,343422.86,2014-04-17,Closed,0


In [31]:
df = df.drop_duplicates(subset='loan_id', keep='first')


In [32]:
df.shape

(3500, 7)

In [37]:
#loan.csv → fix negative loan_amount
df['loan_amount'] = df['loan_amount'].abs()

In [38]:
df[df['loan_amount'] < 0]

,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud


In [39]:
df.dtypes

loan_id                 int64
customer_id             int64
loan_type              object
loan_amount           float64
issue_date     datetime64[ns]
loan_status            object
is_fraud                int64
dtype: object

In [40]:
df.head(179)

,loan_id,customer_id,loan_type,loan_amount,issue_date,loan_status,is_fraud
0,1,4015,Business,558811.91,2022-04-21,Default,0
1,2,4007,Education,370193.71,2014-05-28,Closed,0
2,3,4028,Business,344548.54,2025-09-29,Active,0
3,4,4035,Home,692764.55,2015-01-13,Closed,0
4,5,4040,Education,746639.94,2025-01-07,Closed,0
...,...,...,...,...,...,...,...
174,175,4007,Personal,481220.55,2018-04-07,Closed,0
175,176,4010,Home,384959.20,2024-12-10,Active,0
176,177,4035,Business,526196.11,2019-09-29,Closed,0
177,178,4035,Gold,219103.82,2024-05-20,Active,0


In [41]:
df.shape


(3500, 7)

In [42]:
df.to_csv(r"C:\Users\swath\Downloads\loan.csv", index=False)